In [0]:
processName = dbutils.widgets.get("prm_processName")

nextSourceFileDateSql = f"""select NVL(max(PROCESS_FILE_TABLE_DATE)+1, "2023-01-01") as NEXT_SOURCE_FILE_DATE from pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS where PROCESS_NAME='{processName}' and PROCESS_STATUS='Completed'"""

In [0]:
nextSourceFileDateDf = spark.sql(nextSourceFileDateSql)


In [0]:
type(nextSourceFileDateDf.select("NEXT_SOURCE_FILE_DATE").collect()[0])

In [0]:
from datetime import datetime

dailyPricingSourceBaseURL = "https://retailpricing.blob.core.windows.net/"
dailyPricingSourceFileDate = datetime.strptime(str(nextSourceFileDateDf.select("NEXT_SOURCE_FILE_DATE").collect()[0]["NEXT_SOURCE_FILE_DATE"]), "%Y-%m-%d").strftime("%d%m%Y")
dailyPricingSourceFile = f"PW_MW_DR_{dailyPricingSourceFileDate}.csv"
dailyPricingSourceFolder = "daily-pricing/"

dailyPricingSinkLayerName = "bronze"
dailyPricingSinkStorageAccountName = "adlsudadalakehousedev"
dailyPricingSinkFolderName = "daily-pricing"


In [0]:
import pandas as pds

In [0]:
dailyPricingSourceURL = dailyPricingSourceBaseURL + dailyPricingSourceFolder + dailyPricingSourceFile 


In [0]:
dailyPricingPandasDF = pds.read_csv(dailyPricingSourceURL)

In [0]:
 dailyPricingPandasDF

In [0]:
dailyPricingSparkDF = spark.createDataFrame(dailyPricingPandasDF)

In [0]:
dailyPricinSinkFolderPath = f"abfss://{dailyPricingSinkLayerName}@{dailyPricingSinkStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"

In [0]:
print(dailyPricinSinkFolderPath)

In [0]:
from pyspark.sql.functions import current_timestamp
(
    dailyPricingSparkDF.
    withColumn("source_file_load_date",current_timestamp()).
    write.
    mode("append").
    option("header","true").
    csv(dailyPricinSinkFolderPath)
)

In [0]:
processName = 'dailyPricingSourceIngest'
processStatus = "Completed"

processInsertSql = f"INSERT INTO pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS(PROCESS_NAME, PROCESS_FILE_TABLE_DATE, PROCESS_STATUS) VALUES ('{processName}','{nextSourceFileDateSql}','{processStatus}')"


In [0]:
%sql
alter table pricing_analytics.processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS
add columns(PROCESSED_TABLE_DATE_TIME timestamp)